In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import numpy as np
import pandas as pd

torch.__version__
import random

In [6]:
BATCH_SIZE = 100

traindb=datasets.MNIST('data', train=True, download=True, #cifar10数据集是经典数据集，可以直接载入，第一次使用需要下载。
                       transform=transforms.Compose([
                           transforms.ToTensor(), # 转换成torch.tensor
                       ]))

train_db, val_db, drop_db= torch.utils.data.random_split(traindb, [8000, 2000,50000]) #这里将数据集分成了三份，为了看网络在小样本数据下的性能，
#（接上一行）这里将40000个数据直接扔掉了，5000个训练数据，5000个验证数据，这三个加起来，要等于总的数据50000个。
# train_db, val_db= torch.utils.data.random_split(traindb, [40000, 10000])

print('db1:', len(train_db), 'db2:', len(val_db))
train_loader = torch.utils.data.DataLoader(   #搞出一个训练集
    train_db,
    batch_size=BATCH_SIZE, shuffle=True)
val_loader = torch.utils.data.DataLoader(     #搞出一个验证集
    val_db,
    batch_size=BATCH_SIZE, shuffle=True)

print(len(train_loader))

db1: 8000 db2: 2000
80


In [7]:
class customF(nn.Module): #咱们的激活函数，阈值+噪声


    def __init__(self, width):
        super().__init__()
        self.sigma = nn.Parameter(torch.rand(1, width, 1, 1)) #两个可学习参数 sigma 和 theta
        self.theta = nn.Parameter(torch.rand(1, width, 1, 1))

    def forward(self, input):
      #  temp1=self.sigma*torch.exp(-input*input/2/self.sigma/self.sigma)/2.506628
        temp=0.5-torch.erf((self.theta-input)/self.sigma/1.4142136)/2 #前向传播过程
      #  return temp1+temp2
        return temp

In [8]:
class customFC(nn.Module): #咱们的激活函数，阈值+噪声


    def __init__(self, width):
        super().__init__()
        self.sigma = nn.Parameter(torch.randn(1, width)) #两个可学习参数 sigma 和 theta
        self.theta = nn.Parameter(torch.randn(1, width))

    def forward(self, input):
      #  temp1=self.sigma*torch.exp(-input*input/2/self.sigma/self.sigma)/2.506628
        temp=0.5-torch.erf((self.theta-input)/self.sigma/1.4142136)/2 #前向传播过程
      #  return temp1+temp2
        return temp

In [9]:
class ConvNet(nn.Module): 
    def __init__(self):
        super().__init__()

        self.convdense = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1),
            nn.BatchNorm2d(16)
            
        )
        self.customdense = customF(16)

        self.fc1 = nn.Linear(3136, 256) 
        self.customFc1 = customFC(256)
        self.fc2 = nn.Linear(256, 10) 
    def forward(self,x):
        in_size = x.size(0) 
        out = self.convdense(x)
        
        out = self.customdense(out) 
        
        out = torch.max_pool2d(out, 2, 2) 

               
        out = out.view(in_size, -1) 
        out = self.fc1 (out) 
        out = self.customFc1(out) 
        out = self.fc2(out) 
        out = F.log_softmax(out, dim=1) 
        return out

In [109]:
class ConvNet(nn.Module): #激活函数为我们构造的激活函数。
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 16, 3, padding=1) 
        self.bn1 = nn.BatchNorm2d(16)
        self.custom1 = customF(16)
    
        #self.conv2 = nn.Conv2d(16, 32, 3,padding=1) 
        #self.custom2 = customF(32)
     
        #self.fc1 = nn.Linear(3136, 256) 
       # self.customFc1 = customFC(256)
        self.fc2 = nn.Linear(3136, 10) 
    def forward(self,x):
        in_size = x.size(0) 
        out = self.conv1(x) 
        out = self.bn1(out)
        out = F.max_pool2d(out, 2, 2)     
        out = self.custom1(out,20) 
        
        
        #out = self.conv2(out) 
       # out = self.custom2(out,32) 
        #out = F.max_pool2d(out, 2, 2)
               
        out = out.view(in_size, -1) 
        #out = self.fc1(out) 
        #out = self.customFc1(out)
         
        out = self.fc2(out) 
        out = F.log_softmax(out, dim=1) 
        return out

In [49]:
class ConvNet(nn.Module): #激活函数为 Sigmoid 函数  最后分类层为softmax。
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 16, 3, padding=1) 
        self.bn1 = nn.BatchNorm2d(16)
        #self.custom1 = customF(16)  
    
        #self.conv2 = nn.Conv2d(6, 32, 3,padding=1) 
        #self.custom2 = customF(32)
     
        self.fc1 = nn.Linear(3136, 256) 
        self.fc2 = nn.Linear(256, 10) 
    def forward(self,x):
        in_size = x.size(0) 
        out = self.conv1(x) 
        out = self.bn1(out) 
        out = F.sigmoid(out)
        out = F.max_pool2d(out, 2, 2) 
        
        
        #out = self.conv2(out) 
       # out = self.custom2(out,32) 
        #out = F.max_pool2d(out, 2, 2)
               
        out = out.view(in_size, -1) 
        out = self.fc1(out) 
        out = F.sigmoid(out)
        #out = F.relu(out) 
        out = self.fc2(out) 
        out = F.log_softmax(out, dim=1) 
        return out

In [40]:
class ConvNet(nn.Module): ##激活函数为 Relu 函数  最后分类层为softmax。
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 16, 3, padding=1) 
        self.bn1 = nn.BatchNorm2d(16)
        #self.custom1 = customF(16)
    
        #self.conv2 = nn.Conv2d(6, 32, 3,padding=1) 
        #self.custom2 = customF(32)
     
        self.fc1 = nn.Linear(3136, 256) 
        self.fc2 = nn.Linear(256, 10) 
    def forward(self,x):
        in_size = x.size(0) 
        out = self.conv1(x) 
        out = self.bn1(out) 
        out = F.relu(out)
        out = F.max_pool2d(out, 2, 2) 
        
        
        #out = self.conv2(out) 
       # out = self.custom2(out,32) 
        #out = F.max_pool2d(out, 2, 2)
               
        out = out.view(in_size, -1) 
        out = self.fc1(out) 
        out = F.relu(out) 
        out = self.fc2(out) 
        out = F.log_softmax(out, dim=1) 
        return out

In [10]:
SEED = 600

torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
np.random.seed(SEED)
random.seed(SEED)
torch.backends.cudnn.deterministic = True



EPOCHS=100# 总共训练批次
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
#DEVICE = ("cpu")
model = ConvNet().to(DEVICE)
#model = ConvNet().cuda()
#optimizer = optim.Adam(model.parameters(), lr=0.001,betas=(0.96,0.96))
optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum=0.8) #这里用的是SGD算法

In [11]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        para1[epoch-1]=loss.data
        
       # if(batch_idx+1)%125 == 0:   #训练集5000张图片，batchsize=64，每epoch迭代约78次，可以39次迭代验证一次。
       #         print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        #        epoch, batch_idx * len(data), len(train_loader.dataset),                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
        #        100. * batch_idx / len(train_loader), loss.item()))
        
   # print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch,loss.item()))                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    loss.item()))
    print(epoch,loss.item())
    
    valid(model, DEVICE, val_loader)

In [12]:
def valid(model, device, valid_loader):
    model.eval()
    valid_loss = 0
    valid_correct = 0
    with torch.no_grad():
        for data, target in valid_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            valid_loss += F.nll_loss(output, target, reduction='sum').item() # 将一批的损失相加
            pred = output.max(1, keepdim=True)[1] # 找到概率最大的下标
            valid_correct += pred.eq(target.view_as(pred)).sum().item()

    valid_loss /= len(valid_loader.dataset)
    para2[epoch-1]= valid_correct
    print('Valid set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
         valid_loss, valid_correct, len(val_loader.dataset),
         100. * valid_correct / len(val_loader.dataset)))

In [13]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # 将一批的损失相加
            pred = output.max(1, keepdim=True)[1] # 找到概率最大的下标
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    #para3[epoch-1]=test_loss.data
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [14]:
para1=torch.zeros(EPOCHS)
para2=torch.zeros(EPOCHS)

para3=torch.tensor([])


for epoch in range(1, EPOCHS + 1):
    train(model, DEVICE, train_loader, optimizer, epoch)
    if epoch%1==0:
        for name, param in model.named_parameters(): #在所有的可学习参数里
            if param.requires_grad:
                if name=='custom1.sigma': #找到sigma
                    temppara1=param.data  #记下来
                    a=temppara1.reshape(-1,1)
                   # print(a)
                    para3=torch.cat([para3,a],dim=0) #存到para1的末尾

    
    
#test(model, DEVICE, val_loader)

TypeError: forward() missing 1 required positional argument: 'width'

In [ ]:
expCorrect = 0 #这个程序段实现了实验测试，val_db为测试集，用的激活函数是M个子系统并联，加入高斯噪声的

picNumber = len(val_db)  # 测试数据的个数
M=3  # 并联阈值函数的个数
nodeN=16   # 第一层卷积核的个数
nodeFC=3136
picSample,_ = val_db[0]
height = picSample.size()[1]  # 提取图片的高
width = picSample.size()[2]   # 提取图片的宽

model.eval()
with torch.no_grad():
    for kk in range (picNumber):

        tempimg, templabel= val_db[kk]  # 提取测试图片及标签

        label2=torch.tensor(templabel, dtype=float)
        label2 = torch.unsqueeze(label2,dim=0) 

        tempimg2 = torch.unsqueeze(tempimg,dim=0)

        outAfterConv=model.convdense(tempimg2.to(DEVICE)) #xx,16,28,28

    
        tempAfterCustom1=torch.zeros(1, nodeN, height, width).to(DEVICE)
        for jj in range (nodeN):     
            fig1 = outAfterConv[:,jj,:,:].expand(1, M, height, width)
            noiseEta = model.customdense.sigma[0,jj,0,0]*torch.randn(1,M, height, width).to(DEVICE)
            thres = model.customdense.theta[0,jj,0,0].to(DEVICE)
            fig2 = 0.5*(torch.sign(fig1+noiseEta+thres)+torch.sign(fig1+noiseEta-thres))
            #fig2 = torch.sign(fig1+noiseEta-thres)
            fig3 = torch.mean(fig2,dim=1,keepdim=True)
            tempAfterCustom1[:,jj,:,:] = fig3.view(1, height, width)

        outAfterCustom1 = tempAfterCustom1
        outAfterPool=torch.max_pool2d(outAfterCustom, 2, 2)
        outBeforeFc = outAfterPool.view(1, -1) 
        
        tempAfterCustom1=torch.zeros(nodeFC, 1).to(DEVICE)
        for jj in range (nodeFC):     
            noiseEta = model.customdense.sigma[0,jj,0,0]*torch.randn(1,M, height, width).to(DEVICE)
            thres = model.customdense.theta[0,jj,0,0].to(DEVICE)
            fig2 = 0.5*(torch.sign(fig1+noiseEta+thres)+torch.sign(fig1+noiseEta-thres))
            #fig2 = torch.sign(fig1+noiseEta-thres)
            fig3 = torch.mean(fig2,dim=1,keepdim=True)
            tempAfterCustom1[:,jj,:,:] = fig3.view(1, height, width)

        
        
        outAfterFc1 = model.fc1(outBeforeFc)
        
        
        outAfterFc = model.fc2(outBeforeFc)

        outExp=F.log_softmax(outAfterFc, dim=1)

        expPred = outExp.max(1,keepdim=True)[1]
        expCorrect += expPred.eq(label2.view_as(expPred).to(DEVICE)).sum().item()
#    print(expCorrect/picNumber)

print('Experimental Test Accuracy: {}/{} ({:.2f}%)\n'.format(expCorrect, picNumber,
        100. * expCorrect / picNumber))

In [ ]:
print(para3)   #储存每一步的sigma
frame1=pd.DataFrame(para3.data.numpy()) 
frame1.to_csv('C:/results/sigma.csv',index=False)

In [10]:
#注意！上面的程序用验证集充当测试集。如果要用mnist的测试集，用下面的代码，然后把上一个模块中的测试代码test(model, DEVICE, val_loader)变成tr

In [48]:
frame1=pd.DataFrame(para1.data.numpy()) 
frame1.to_csv('C:/results/loss_train011.csv',index=False)
frame1=pd.DataFrame(para2.data.numpy()) 
frame1.to_csv('C:/results/valid_correct011.csv',index=False)


In [14]:
frame1=pd.DataFrame(para1.data.numpy()) 
frame1.to_csv('C:results/loss_traindl.csv',index=False)
frame1=pd.DataFrame(para2.data.numpy()) 
frame1.to_csv('C:results/valid_correctdl.csv',index=False)

In [21]:
for name, param in model.named_parameters():
    print (name)

conv1.weight
conv1.bias
bn1.weight
bn1.bias
custom1.sigma
custom1.theta
fc1.weight
fc1.bias
customFc1.sigma
customFc1.theta
fc2.weight
fc2.bias


In [24]:
frame1=pd.DataFrame(model.fc1.bias.data.numpy())  #存每个epoch的sigma和theta

frame1.to_csv('C:/results/fc1bias.csv',index=False) 
frame2=pd.DataFrame(model.fc1.weight.data.numpy())  #存每个epoch的sigma和theta
frame2.to_csv('C:/results/fc1weight.csv',index=False) 
frame3=pd.DataFrame(model.fc2.bias.data.numpy())  #存每个epoch的sigma和theta
frame3.to_csv('C:/results/fc2bias.csv',index=False) 
frame4=pd.DataFrame(model.fc2.weight.data.numpy())  #存每个epoch的sigma和theta
frame4.to_csv('C:/results/fc2weight.csv',index=False) 
frame5=pd.DataFrame(model.bn1.bias.data.numpy())  #存每个epoch的sigma和theta
frame5.to_csv('C:/results/bn1bias.csv',index=False) 
frame6=pd.DataFrame(model.bn1.weight.data.numpy())  #存每个epoch的sigma和theta
frame6.to_csv('C:/results/bn1weight.csv',index=False) 


In [20]:
a = model.conv1.weight.detach().numpy()  #存所有的卷积核
b = a.reshape(-1,9) #把三维的卷积核转化为9*16的矩阵进行存储
np.savetxt('conv1weight.csv',b,delimiter=',')

a = model.conv1.bias.detach().numpy() #存卷积层的偏执
b = a.reshape(-1,1)
np.savetxt('conv1bias.csv',b,delimiter=',')


a = model.custom1.sigma.detach().numpy()  # 存归一化的系数
b = a.reshape(-1,1)
np.savetxt('custom1sigma.csv',b,delimiter=',')

a = model.custom1.theta.detach().numpy()
b = a.reshape(-1,1)
np.savetxt('custom1theta.csv',b,delimiter=',')

In [22]:
print (model.customFc1.sigma)

Parameter containing:
tensor([[-0.5915,  2.0098, -0.4391, -0.2575, -1.1112, -0.8826, -0.2765, -0.9498,
         -0.0124,  0.4396,  1.0969, -1.0753, -0.2512,  0.0423,  0.0092, -3.0388,
          0.0384,  0.0942, -0.6420,  1.1186, -0.4892, -1.2306,  0.0102, -0.4254,
          0.4124,  0.6332,  0.1364,  1.2235,  1.7544,  0.1135,  0.2693,  0.0080,
          0.0132,  0.0093,  0.7880,  0.4676,  0.1194, -0.0104, -0.0487, -0.5085,
         -1.9784, -1.0716,  1.1823, -0.8316,  0.5135, -0.9583,  0.4216, -0.0114,
          1.5141, -1.4576,  0.5765, -0.2202, -0.1937, -0.0082,  0.9217, -0.0119,
         -1.3005,  0.4570, -1.2840,  1.0853,  0.0068, -0.1302, -0.0871, -0.0089,
         -4.0709, -0.9054, -0.3224, -0.8944,  1.5545, -0.0063, -0.6245, -1.3657,
         -1.2551,  0.4388,  1.0861,  0.0619, -0.5314,  0.5670,  0.2596,  0.4339,
          1.8044,  0.7537,  0.9441, -0.5992, -0.2753,  0.0112,  0.1684, -1.5292,
         -0.0817, -0.3967, -0.0768,  0.6471, -0.4027, -0.3745,  1.4199, -0.0794,
      

In [26]:
print (model.conv1.weight)

Parameter containing:
tensor([[[[ 0.0052,  0.1707,  0.2916],
          [-0.2794, -0.3592, -0.1685],
          [ 0.2802, -0.0667,  0.0333]]],


        [[[ 0.2643, -0.3754,  0.0663],
          [-0.0070, -0.0650,  0.0880],
          [ 0.0579, -0.2710,  0.1737]]],


        [[[ 0.3234,  0.1599,  0.3626],
          [ 0.0989,  0.0285, -0.1436],
          [-0.2442, -0.1727, -0.1506]]],


        [[[ 0.2296, -0.1686, -0.2508],
          [ 0.1457, -0.1463, -0.1180],
          [-0.2138, -0.1083, -0.1363]]],


        [[[-0.0337, -0.2428, -0.0622],
          [-0.3229, -0.1352,  0.0137],
          [ 0.3229,  0.0325,  0.1408]]],


        [[[-0.2333,  0.0322, -0.1510],
          [-0.3055,  0.2478,  0.0134],
          [-0.2244, -0.2070,  0.1933]]],


        [[[-0.2787,  0.2789, -0.1164],
          [ 0.1156, -0.0906, -0.2311],
          [ 0.1849,  0.1853,  0.2738]]],


        [[[-0.3713, -0.2860, -0.1522],
          [-0.2762,  0.1165, -0.3647],
          [ 0.2344,  0.0846, -0.1037]]],


        [[